In [1]:
from dotenv import load_dotenv
import os
load_dotenv(override=True) # take environment variables from .env.
ai_foundry_endpoint = os.getenv("AI_FOUNDRY_ENDPOINT")
doc_agent_id = os.getenv("DOC_AGENT_ID") #Optional. Gets the agent that only has the documentation.
op_agent_id = os.getenv("OP_AGENT_ID")

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder

print("test")

project = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=ai_foundry_endpoint)

doc_agent = project.agents.get_agent(doc_agent_id)
op_agent = project.agents.get_agent(op_agent_id)

test


In [3]:
thread = project.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_rItZTzSk9xaXJKTraDKaGzE2


In [9]:
message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="How to deploy pods?"
)

In [10]:
run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=doc_agent.id)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

    for message in messages:
        if message.text_messages:
            print(f"{message.role}: {message.text_messages[-1].text.value}")

MessageRole.USER: What's all the pods in the default namespace?
MessageRole.AGENT: It seems I'm unable to access the tools needed to provide you the information about pods in the default namespace. Please try again later or use a different method to retrieve that information.
MessageRole.USER: How to deploy pods?
MessageRole.AGENT: To deploy pods in Kubernetes, you typically use a YAML manifest file that describes the desired state of the pods. Here are the steps to deploy a pod:

1. **Create a YAML Manifest File**: Define the pod specifications in a YAML file. For example, create a file called `my-pod.yaml` with the following content:

   ```yaml
   apiVersion: v1
   kind: Pod
   metadata:
     name: my-pod
   spec:
     containers:
     - name: my-container
       image: nginx
       ports:
       - containerPort: 80
   ```

   In this example:
   - **apiVersion**: Version of the Kubernetes API to use.
   - **kind**: Specifies the type of object (Pod).
   - **metadata**: Contains the

In [11]:
def list_all_threads():
    """
    Lists all active conversation threads for the project.
    """
    print("Listing all active threads...")
    try:
        threads = project.agents.threads.list()
        if not threads:
            print("No active threads found.")
        else:
            for thread in threads:
                print(f" - Thread ID: {thread.id}")
        print("-" * 20)
    except Exception as e:
        print(f"An error occurred while listing threads: {e}")

In [12]:
list_all_threads()

Listing all active threads...
 - Thread ID: thread_rItZTzSk9xaXJKTraDKaGzE2
 - Thread ID: thread_nio6rIFGQnzuf0M0JcE914Gy
 - Thread ID: thread_eCr5qY0ynTj1VVCRWPHAN3EC
 - Thread ID: thread_PkbTEPkUAR6sMIEt7sRV25WA
 - Thread ID: thread_cn4SZlZHaMTjI2CXmT0jxNCJ
 - Thread ID: thread_yyG37XrC4QLR1lQQC1GbwUqT
 - Thread ID: thread_Fblchzyv4ij4HHJXa5jIwno2
 - Thread ID: thread_gSenkwbmHRQmKNh0ilEp0kf7
 - Thread ID: thread_e7tr2NLmJHJDYbUX5Vmswl0k
 - Thread ID: thread_RRBelROjq0iiku52MWU3jXe1
 - Thread ID: thread_qpat5DHhIFsvafNSMcHGvQlV
 - Thread ID: thread_MnqtONdoLdL6soCUf7Hd40nF
 - Thread ID: thread_PDtMTWi7phbuyoiQGMDCj7Vl
 - Thread ID: thread_A4WrD8QrieL5y4jZ1o64RBoD
 - Thread ID: thread_qvL15z2ClCP6PEkOpbJil8CE
 - Thread ID: thread_1ImGDCxIuQ5ZUVwxLMCS80ch
 - Thread ID: thread_5HmPXh1hvHRXzKL4UyH8yzQc
--------------------


In [13]:
def delete_all_threads():
    print("Deleting all active threads...")
    try:
        threads = project.agents.threads.list()
        if not threads:
            print("No active threads found.")
        else:
            for thread in threads:
                if thread.id != "thread_5HmPXh1hvHRXzKL4UyH8yzQc":
                    project.agents.threads.delete(thread.id)
        print("-" * 20)
    except Exception as e:
        print(f"An error occurred while deleting threads: {e}")

In [14]:
delete_all_threads()

Deleting all active threads...
--------------------


# Add tools

In [15]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from app import list_kubernetes_pods, restart_all_pods

op_agent.tools = [list_kubernetes_pods, restart_all_pods]
op_agent.update(op_agent)

for tool in op_agent.tools:
    print(f"Tool name: {tool.name}")
    print(f"Description: {tool.description}")
    print("---")

Tool name: list_kubernetes_pods
Description: A tool to list all pods in a given Kubernetes namespace (default: "default"). ONLY use when the user is clearly requesting an action like 'list pods' or 'please list pods'. DO NOT use when the user is asking for an explanation or instructions


Args:
    namespace (str): The namespace to list pods from. Defaults to "default".

Returns:
    str: A formatted list of pod names and statuses, or an error message.
---
Tool name: restart_all_pods
Description: Restart all pods in a given Kubernetes namespace. ONLY use when the user is clearly requesting an action like 'restart pods'.

Args:
    namespace (str): Namespace in which to restart all pods. Defaults to "default".
    confirm (bool): If True, actually deletes the pods. If False, just previews which pods would be restarted.

Returns:
    str: Preview of pods to be restarted, or success/error message.
---


In [5]:
# The decorator returns a Tool instance
tool_instance = list_kubernetes_pods

# Invoke it directly
result = tool_instance.invoke("What's all the pods in the environment?")
print(result)

Kubernetes configuration not found. Please ensure your ~/.kube/config file is set up correctly.


In [7]:
message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's all the pods in the default namespace?"
)

In [8]:
run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=op_agent.id)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

    for message in messages:
        if message.text_messages:
            print(f"{message.role}: {message.text_messages[-1].text.value}")

MessageRole.USER: What's all the pods in the default namespace?
MessageRole.AGENT: It seems I'm unable to access the tools needed to provide you the information about pods in the default namespace. Please try again later or use a different method to retrieve that information.
